In [1]:
print(f"Current working directory: {os.getcwd()}")


NameError: name 'os' is not defined

In [2]:
import zipfile
import pandas as pd

In [3]:
from google.colab import files
import zipfile
import os

# Upload files interactively using files.upload()
uploaded = files.upload()

# Assuming the uploaded file is named 'dataset and all.zip'
zip_file_name = next(iter(uploaded))
zip_file_path = zip_file_name  # The uploaded file path in Colab

# Define the extraction path
extract_to_path = '/content/extracted_files/'  # Adjust this path as needed

def unzip_file(zip_file, extract_to):
    try:
        # Check if the ZIP file exists
        if not os.path.isfile(zip_file):
            print(f"The file {zip_file} does not exist.")
            return

        # Create the extraction directory if it does not exist
        if not os.path.exists(extract_to):
            os.makedirs(extract_to)

        # Unzipping the file
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_to)

        # Listing the extracted files
        extracted_files = os.listdir(extract_to)
        print(f"Files extracted to {extract_to}:")
        print(extracted_files)

    except FileNotFoundError:
        print(f"Error: The file {zip_file} does not exist.")

    except zipfile.BadZipFile:
        print(f"Error: {zip_file} is not a valid ZIP file.")

    except Exception as e:
        print(f"An error occurred while extracting {zip_file}: {str(e)}")

# Example usage:
if __name__ == "__main__":
    unzip_file(zip_file_path, extract_to_path)


Saving dataset and all.zip to dataset and all.zip
Files extracted to /content/extracted_files/:
['training_set_labels.csv', 'submission_format.csv', 'training_set_features.csv', 'test_set_features.csv']


In [4]:
def display_first_few_rows(extracted_files, extract_to):
    try:
        # Assuming the specific CSV file names
        features_csv = 'training_set_features.csv'
        labels_csv = 'training_set_labels.csv'

        # Check if the files exist in the extracted directory
        if features_csv in extracted_files and labels_csv in extracted_files:
            # Load CSV files into Pandas DataFrames
            train_features = pd.read_csv(os.path.join(extract_to, features_csv))
            train_labels = pd.read_csv(os.path.join(extract_to, labels_csv))

            # Display first few rows of each DataFrame
            print("First few rows of features DataFrame:")
            print(train_features.head())

            print("\nFirst few rows of labels DataFrame:")
            print(train_labels.head())

        else:
            print(f"Error: Required CSV files ({features_csv} and {labels_csv}) not found in {extract_to}.")

    except Exception as e:
        print(f"An error occurred while displaying first few rows: {str(e)}")

# Example usage:
if __name__ == "__main__":
    # Unzip the uploaded ZIP file
    extracted_files = unzip_file(zip_file_path, extract_to_path)

    # If extraction was successful, display first few rows of CSV files
    if extracted_files:
        display_first_few_rows(extracted_files, extract_to_path)

Files extracted to /content/extracted_files/:
['training_set_labels.csv', 'submission_format.csv', 'training_set_features.csv', 'test_set_features.csv']


In [5]:
import pandas as pd

# Assuming the paths to your CSV files in Colab
features_csv_path = '/content/extracted_files/training_set_features.csv'
labels_csv_path = '/content/extracted_files/training_set_labels.csv'

# Load training set features and labels into Pandas DataFrames
train_features = pd.read_csv(features_csv_path)
train_labels = pd.read_csv(labels_csv_path)

# Display basic information about features DataFrame
print("Training Set Features:")
print("Number of rows and columns:", train_features.shape)
print("Column names:", train_features.columns.tolist())
print("\nData types:")
print(train_features.dtypes)
print("\nSummary statistics:")
print(train_features.describe())

# Display basic information about labels DataFrame
print("\nTraining Set Labels:")
print("Number of rows and columns:", train_labels.shape)
print("Column names:", train_labels.columns.tolist())
print("\nData types:")
print(train_labels.dtypes)
print("\nSummary statistics:")
print(train_labels.describe())


Training Set Features:
Number of rows and columns: (26707, 36)
Column names: ['respondent_id', 'xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months', 'health_worker', 'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'household_adults', 'household_children', 'employment_industry', 'employment_occupation']

Data types:
respondent_id                    int64
xyz_concern                    float64
xyz_knowledge                  float64
behavioral_antiviral_meds      

In [6]:
# Display all rows and columns of the labels DataFrame
print("Training Set Labels:")
with pd.option_context('display.max_columns', None):  # Show all columns
    with pd.option_context('display.max_rows', None):  # Show all rows
        print(train_labels)

Streaming output truncated to the last 5000 lines.
21707          21707            0                 0
21708          21708            1                 1
21709          21709            0                 0
21710          21710            0                 0
21711          21711            0                 0
21712          21712            0                 0
21713          21713            0                 0
21714          21714            1                 1
21715          21715            0                 0
21716          21716            1                 1
21717          21717            0                 0
21718          21718            0                 0
21719          21719            1                 1
21720          21720            0                 0
21721          21721            0                 0
21722          21722            0                 1
21723          21723            0                 0
21724          21724            0                 0
21725        

In [7]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Load the training set features CSV file (adjust path as necessary)
features_csv_path = '/content/extracted_files/training_set_features.csv'
train_features = pd.read_csv(features_csv_path)

# Separate features into numeric and categorical
numeric_features = train_features.select_dtypes(include=['float64']).columns
categorical_features = train_features.select_dtypes(include=['object']).columns

# Preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())])  # Standardize features by removing mean and scaling to unit variance

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])  # Impute missing values with most frequent value

# Combining preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocess the features
train_features_preprocessed = preprocessor.fit_transform(train_features)

# Extract transformed numeric and categorical feature names
processed_feature_columns = list(numeric_features) + list(categorical_features)

# Create a DataFrame of preprocessed features (for verification)
train_features_preprocessed_df = pd.DataFrame(train_features_preprocessed, columns=processed_feature_columns)

# Display the first few rows of preprocessed features
print("First few rows of preprocessed features:")
print(train_features_preprocessed_df.head())

# Verify preprocessing steps by comparing statistics before and after preprocessing
print("\nBefore preprocessing:")
print(train_features.describe())

print("\nAfter preprocessing:")
print(train_features_preprocessed_df.describe())




First few rows of preprocessed features:
  xyz_concern xyz_knowledge behavioral_antiviral_meds behavioral_avoidance  \
0   -0.681849     -2.044279                 -0.226293            -1.634957   
1    1.518373      1.197027                 -0.226293             0.611637   
2   -0.681849     -0.423626                 -0.226293             0.611637   
3   -0.681849     -0.423626                 -0.226293             0.611637   
4    0.418262     -0.423626                 -0.226293             0.611637   

  behavioral_face_mask behavioral_wash_hands behavioral_large_gatherings  \
0            -0.272097             -2.177944                    -0.74589   
1            -0.272097              0.459149                    -0.74589   
2            -0.272097             -2.177944                    -0.74589   
3            -0.272097              0.459149                     1.34068   
4            -0.272097              0.459149                     1.34068   

  behavioral_outside_home behavio

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming train_features_processed and train_labels are already defined

# Align features and labels by setting respondent_id as index
train_features.set_index('respondent_id', inplace=True)
train_labels.set_index('respondent_id', inplace=True)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Verify the split
print("Shape of training features:", X_train.shape)
print("Shape of validation features:", X_val.shape)
print("Shape of training labels:", y_train.shape)
print("Shape of validation labels:", y_val.shape)


Shape of training features: (21365, 35)
Shape of validation features: (5342, 35)
Shape of training labels: (21365, 2)
Shape of validation labels: (5342, 2)


In [9]:
import pandas as pd
import numpy as np  # Import NumPy
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

# Load the training set features CSV file (adjust path as necessary)
features_csv_path = '/content/extracted_files/training_set_features.csv'
train_features = pd.read_csv(features_csv_path)

# Separate features into numeric and categorical
numeric_features = train_features.select_dtypes(include=['float64']).columns
categorical_features = train_features.select_dtypes(include=['object']).columns

# Preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())])  # Standardize features by removing mean and scaling to unit variance

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Encode categorical variables

# Combining preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocess the features
train_features_preprocessed = preprocessor.fit_transform(train_features)

# Set respondent_id as index for train_features (assuming it exists now)
train_features.set_index('respondent_id', inplace=True)

# Load the training set labels CSV file (adjust path as necessary)
labels_csv_path = '/content/extracted_files/training_set_labels.csv'
train_labels = pd.read_csv(labels_csv_path)

# Set respondent_id as index for train_labels
train_labels.set_index('respondent_id', inplace=True)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_features_preprocessed, train_labels, test_size=0.2, random_state=42)

# Train MultiOutputClassifier with RandomForestClassifier
model = MultiOutputClassifier(RandomForestClassifier(random_state=42))
model.fit(X_train, y_train)

# Evaluate on validation set
y_pred = model.predict_proba(X_val)
y_pred = np.array([pred[:, 1] for pred in y_pred]).T  # Get probabilities for class 1

roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], y_pred[:, 0])
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], y_pred[:, 1])

overall_score = (roc_auc_xyz + roc_auc_seasonal) / 2
print('Overall ROC AUC Score:', overall_score)

# Assuming test set features are loaded similarly as train_features (adjust path as necessary)
test_features_csv_path = '/content/extracted_files/test_set_features.csv'
test_features = pd.read_csv(test_features_csv_path)

# Preprocess test set features
test_features_processed = preprocessor.transform(test_features)

# Make predictions on test set
test_predictions = model.predict_proba(test_features_processed)
test_predictions = np.array([pred[:, 1] for pred in test_predictions]).T  # Get probabilities for class 1

# Create submission DataFrame
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],  # Assuming 'respondent_id' is present in test_features
    'xyz_vaccine': test_predictions[:, 0],
    'seasonal_vaccine': test_predictions[:, 1]
})

# Save submission to CSV file
submission.to_csv('/content/extracted_files/submission.csv', index=False)




Overall ROC AUC Score: 0.840671491285722
